In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3,3) # 输入维度是3（句子、单词、词向量），输出维度是3
inputs = [torch.randn(1,3) for _ in range (5)]  # 构建1*3的2维向量
hidden = (torch.randn(1,1,3),torch.randn(1,1,3))

In [4]:
for i in inputs:
    out, hidden = lstm(i.view(1,1,-1), hidden)  # 将输入变形成3维，所有数据都放在第三维
# inputs = torch.cat(inputs).view(len(inputs),1,-1)
# hidden = torch.randn(1,1,3), torch.randn(1,1,3))
print(out)
print(hidden)

tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>)
(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward>))


In [5]:
def prepare_sequence(seq, to_idx):
    idxs = [to_idx[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_idx = {}

for sen, tags in training_data:
    for word in sen:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
            
print(word_to_idx)
tag_to_idx = {"DET": 0, "NN": 1, "V": 2}


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [6]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)  #
        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
#         self.evaluation = F.log_softmax(tagset_size)  # 这个不是类是函数，所以不能预先定义
    
    def forward(self, sentence):
        embeddings = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeddings.view(len(sentence),1,-1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [10]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx))
loss_function = nn.NLLLoss()  # nagetive log likehood

optimizer = optim.SGD(model.parameters(), lr=0.1)

# see tag_scores before traininig
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):
    for sentence, tags in training_data:
        # cause pytorch accumulates gradients
        # we clear grads every time before training
        print(sentence)
        print(tags)
        model.zero_grad()
        
        # prepare data
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)
        
        # input and forward
        tag_scores = model(sentence_in)
        
        # caculate loss,gradients, 
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()  # update optimizers's parameters
        
with torch.no_grad():
    # evaluate the trained model
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    
    tag_scores = model(inputs)
    
    print(tag_scores)

tensor([[-0.6897, -1.3825, -1.3970],
        [-0.8179, -1.2660, -1.2849],
        [-0.8033, -1.2711, -1.3034],
        [-0.7926, -1.3419, -1.2518],
        [-0.8437, -1.2104, -1.3028]])
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', '

['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET'

['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']